<a target="_blank"  href="https://colab.research.google.com/github/rtharungowda/nnc/blob/main/diatom_object_detection.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
</a>

Clone repo with dataset

In [ ]:
%cd /content/
!git clone https://github.com/rtharungowda/nnc.git

In [2]:
!mv nnc Diatom-Non-neuronal-Cognition

# Yolov5 

## Setup




In [ ]:
#clone yolov5 repo
!git clone https://github.com/ultralytics/yolov5.git

#install requirements and download pretrained weights
%cd /content/yolov5/
!pip3 install -r requirements.txt
!sh weights/download_weights.sh

## Train


In [ ]:
%cd /content/

We will be train a yolov5 model on diatom images, all the config files are preset. <br/>
Yolov5 has 4 models
1.   yolov5s (smallest model)
2.   yolov5m
3.   yolov5l
4.   yolov5x (largest model) 
<br/>

change the variables CLF and WEIGHTS depending on which model you want to train

In [ ]:
%%writefile train_yolov5.sh

cd /content/yolov5

#start training
BATCH_SIZE="8" #batchsize
NUM_EPOCHS="600" #number of epochs to train on
TRAIN_YAML="/content/Diatom-Non-neuronal-Cognition/Dataset/30_bbg/related_files/diatom.yaml" #path to config file
CLF="/content/yolov5/models/yolov5s.yaml" #path to model config file
WEIGHTS="/content/yolov5/yolov5s.pt" #path to pretrained weights
DEVICE="0" #change to required device, "cpu" for cpu and "0" or device index for gpu
echo "Batch size : $BATCH_SIZE" 
echo "Number of epochs : $NUM_EPOCHS"
echo "Link to train yaml : $TRAIN_YAML"
echo "Classifier : $CLF"
echo "Using weights from : $WEIGHTS"
echo "Device : $DEVICE"

python3 /content/yolov5/train.py --batch $BATCH_SIZE --epochs $NUM_EPOCHS --data $TRAIN_YAML --cfg $CLF  --weights $WEIGHTS --device $DEVICE


In [ ]:
%pwd
!bash train_yolov5.sh

Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/train/exp/

## Predict 

In [ ]:
%cd /content/

In [ ]:
%%writefile predict_yolov5.sh
#cd to yolov5
cd /content/yolov5

#predict
SOURCE="/content/Diatom-Non-neuronal-Cognition/Dataset/30_bbg/imgs" #path to folder where the images to be predicted are present
WEIGHTS="/content/yolov5/runs/train/exp/weights/best.pt" #path to saved weights
CONF="0.4" #confidance
DEVICE="cpu" #device on which to predict

echo "Source of images : $SOURCE"
echo "Weights from : $WEIGHTS"

echo "Confindance : $CONF"

python3 /content/yolov5/detect.py --source $SOURCE --weights $WEIGHTS --conf $CONF --device $DEVICE

In [ ]:
!sh predict_yolov5.sh

# Scaled Yolov4

## Setup

In [ ]:
%cd /content/

# clone Scaled_YOLOv4
!git clone https://github.com/WongKinYiu/ScaledYOLOv4.git  
%cd /content/ScaledYOLOv4/
#checkout the yolov4-large branch
!git checkout yolov4-large


#install mish activation funciton for cuda
%cd /content/
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python3 setup.py build install

!pip3 install -U PyYAML

#download weights
%cd /content/ScaledYOLOv4/
!pip3 install gdown

!mkdir weights
%cd /content/ScaledYOLOv4/weights/

!gdown https://drive.google.com/u/0/uc?id=1NQwz47cW0NUgy7L3_xOKaNEfLoQuq3EL
!gdown https://drive.google.com/u/0/uc?id=1aXZZE999sHMP1gev60XhNChtHPRMH3Fz

## Train

Replace line *#141* and *#142* in **ScaledYOLOv4/models/yolo.py** to this <br/>
```            
b[:, 4].data += math.log(8 / (640 / s) ** 2)  # obj (8 objects per 640 image)
b[:, 5:].data += math.log(0.6 / (m.nc - 0.99)) if cf is None else torch.log(cf / cf.sum())  # cls
```

In [ ]:
%cd /content/

We will be train a scaled yolov4 model on diatom images, all the config files are preset. <br/>
Yolov4 has 3 models
1.   yolov4-csp (smallest model)    
2.   yolov4-p5 
2.   yolov4-p6
3.   yolov4-p7 (largest model) 
<br/>

change the variables CLF and WEIGHTS depending on which model you want to train

In [ ]:
%%writefile train_scl_yolov4.sh
cd /content/ScaledYOLOv4/

#start training
IMG_SIZE="640" #image size
BATCH_SIZE="8" #batch size
NUM_EPOCHS="600" #number of epochs
TRAIN_YAML="/content/Diatom-Non-neuronal-Cognition/Dataset/30_bbg/related_files/diatom.yaml" #dataset config file path
CLF="/content/ScaledYOLOv4/models/yolov4-p5.yaml" #model config
WEIGHTS="/content/ScaledYOLOv4/weights/yolov4-p5.pt" #pretrained-weights
DEVICE="0" #device
echo "Batch size : $BATCH_SIZE"
echo "Number of epochs : $NUM_EPOCHS"
echo "Link to train yaml : $TRAIN_YAML"
echo "Classifier : $CLF"
echo "Using weights from : $WEIGHTS"
echo "Device : $DEVICE"

python3 /content/ScaledYOLOv4/train.py --img $IMG_SIZE --batch $BATCH_SIZE --epochs $NUM_EPOCHS --data $TRAIN_YAML --cfg $CLF --weights $WEIGHTS  --cache

In [ ]:
!sh train_scl_yolov4.sh

Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir /content/ScaledYOLOv4/runs/exp0/

## Predict

In [ ]:
%cd /content/

In [ ]:
%%writefile predict_scl_yolov4.sh
#cd to yolov4
cd /content/ScaledYOLOv4

#predict
IMG_SIZE="640" #image size
SOURCE="/content/Diatom-Non-neuronal-Cognition/Dataset/30_bbg/imgs" #path to folder of images to make predictions on
WEIGHTS="/content/ScaledYOLOv4/runs/exp0/weights/best.pt" #path to model weights
CONF="0.4" #confidance

echo "Source of images : $SOURCE"
echo "Weights from : $WEIGHTS"
echo "Confindance : $CONF"

python3 /content/ScaledYOLOv4/detect.py --weights $WEIGHTS --img $IMG_SIZE --conf $CONF --source $SOURCE

In [ ]:
!sh predict_scl_yolov4.sh